### 1. The PyNN NeuroML morphology
Let us assume we have a PyNN interfaced NeuroML morphology as shown below.
#### 1.1. For a given NeuromML morphology

In [1]:
from neuroml import Morphology, Segment, Point3DWithDiam as P

In [2]:
soma = Segment(proximal=P(x=0, y=0, z=0, diameter=18.8),
               distal=P(x=18.8, y=0, z=0, diameter=18.8),
               name="soma", id=0)
dend = Segment(proximal=P(x=0, y=0, z=0, diameter=2),
               distal=P(x=-500, y=0, z=0, diameter=2),
               name="dendrite",
               parent=soma, id=1)
neuroml_morph = Morphology(segments=(soma, dend))

#### 1.2. The NeuroML morphology is interface by PyNN 

In [3]:
from pyNN.morphology import NeuroMLMorphology

In [8]:
morph = NeuroMLMorphology(neuroml_morph)

# Problem:
## Given a PyNN NeuroML morphology how does one construct an analogous morphology in [Arbor](https://arbor.readthedocs.io/en/latest/index.html#)

Note that the term morphology in NeuroML does not mean the same as those used by Arbor. In NeuroML the meaning of morphology is that of the organizational formation of the constructed cell. [Morphology in Arbor](https://arbor.readthedocs.io/en/pydoc/morphology.html) describes _cell geometry as unbranched cables with variable radius and associated tree structure_.

In [5]:
import arbor
from arbor import mpoint
from arbor import mnpos

In [17]:
tree = arbor.segment_tree()

In [18]:
# Construct Soma segment in the Tree structure, `0`Arbor-segment
tree.append(mnpos,
            mpoint(morph.segments[0].proximal.x, morph.segments[0].proximal.y, morph.segments[0].proximal.z,
                  morph.segments[0].proximal.diameter/2),
            mpoint(morph.segments[0].distal.x, morph.segments[0].distal.y, morph.segments[0].distal.z,
                  morph.segments[0].distal.diameter/2),
           tag=0)

0

In [20]:
# Attached Dendrite segment to the Tree
tree.append(0, # parent is `0`Arbor-segment
            mpoint(morph.segments[1].proximal.x, morph.segments[1].proximal.y, morph.segments[1].proximal.z,
                  morph.segments[1].proximal.diameter/2),
            mpoint(morph.segments[1].distal.x, morph.segments[1].distal.y, morph.segments[1].distal.z,
                  morph.segments[1].distal.diameter/2),
           tag=3)

1

In [32]:
import re

In [58]:
def create_arbor_tree(nml_morph):
    tree = arbor.segment_tree()
    for i, nml_seg in enumerate(nml_morph):
        append_arbor_tree(tree, nml_seg)
    return tree
    
def append_arbor_tree(tree, nml_seg):
    if not nml_seg.parent:
        tree.append(mnpos, mpoint(nml_seg.proximal.x, nml_seg.proximal.y, nml_seg.proximal.z,
                                  nml_seg.proximal.diameter/2),
                    mpoint(nml_seg.distal.x, nml_seg.distal.y, nml_seg.distal.z,
                           nml_seg.distal.diameter/2), tag=get_swc_tag(nml_seg))
    else:
        tree.append(nml_seg.parent.id,
                    mpoint(nml_seg.proximal.x, nml_seg.proximal.y, nml_seg.proximal.z,
                           nml_seg.proximal.diameter/2),
                    mpoint(nml_seg.distal.x, nml_seg.distal.y, nml_seg.distal.z,
                           nml_seg.distal.diameter/2), tag=get_swc_tag(nml_seg))

def get_swc_tag(nml_seg):
    if re.search("soma", nml_seg.name, re.IGNORECASE):
        return 1
    elif re.search("axon", nml_seg.name, re.IGNORECASE):
        return 2
    elif re.search("dend", nml_seg.name, re.IGNORECASE):
        return 3
    else:
        return 5
        

In [59]:
tree = arbor.segment_tree()

In [60]:
append_arbor_tree(tree, morph.segments[0])
append_arbor_tree(tree, morph.segments[1])

In [62]:
create_arbor_tree(morph.segments)

In [54]:
for i, nml_seg in enumerate(morph.segments):
    print(nml_seg)

<Segment|0|soma>
<Segment|1|dendrite>


In [40]:
re.match("dendrite", "Dendrite", re.IGNORECASE)

<re.Match object; span=(0, 8), match='Dendrite'>

In [41]:
if re.match("dendrite", "Dendrite", re.IGNORECASE):
    print("yes match")
else:
    print("no match")

yes match


In [46]:
if re.search("dend", "apical dendrite", re.IGNORECASE):
    print("yes match")
else:
    print("no match")

yes match
